###### References: 
- https://docs.python.org/3/library/asyncio.html
- Fluent Python, 2nd Edition, by Luciano Ramalho. Chapter 21: Asynchronous Programming

##### Concurrenly Is Not Parrallelism
Concurrency is about dealing with lots of things at once.  
Parallelism is about doing lots of things at once.

#### Native Coroutine
#### Classic Coroutine
#### Generator-based Coroutine
#### Asynchronous  Coroutine


# E.g. : Probing Domains
In terminal:

    python3 blogddom.py

# Awaitable
`await` works for *awaitables*

 * A native coroutine object
 * An `asyncio.Task`
 * An object with an `__await__`  method that returns an iterator
 * Objects written in other languages using `tp_as_async.am_await` function, returning an iterator
 
# Downloading with an asyncio and  HTTPX
In terminal:

    python3 flags_asyncio.py

In [ ]:
def download_many(cc_list: list[str]) -> int:    # needs to be a plain function
    return asyncio.run(supervisor(cc_list))      # execute the event loop

async def supervisor(cc_list: list[str]) -> int:
    async with AsyncClient() as client:          # asynchorous HTTP client operations
        to_do = [download_one(client, cc)
                 for cc in sorted(cc_list)]      # build a list of coroutine objects
        res = await asyncio.gather(*to_do)       # wait for the asyncio.gather coroutine

    return len(res)                              

In [ ]:
import asyncio

from httpx import AsyncClient  

from flags import BASE_URL, save_flag, main 

async def download_one(client: AsyncClient, cc: str):  # must be a native coroutine
    image = await get_flag(client, cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:  # needs to receive the AsyncClient to make the request
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1,
                                  follow_redirects=True)  # returns a ClientResponse object that is also an asynchronous context manager
    return resp.read()  # Network I/O operations are implemented as coroutine methods

# The secret of Native Coroutines : Humble Generators
<img src="await.png" width="75%">

# Asynchronous Context Managers

## PostgreSQL driver:

In [ ]:
tr = connection.transaction()
await tr.start()
try:
    await connection.execute('INSERT INTO mytable VALUES (1, 2, 3)')
except: 
    await tr.rollback()
    raise
else:
    await tr.commit()

In [ ]:
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES(1, 2, 3)")

# Enhancing the asyncio Downloader
In terminal:

    python3 flags2_asyncio.py
    
# Throttling Requests  with a Semaphore

In [ ]:
async def supervisor(cc_list: list[str],
                     base_url: str,
                     verbose: bool,
                     concur_req: int) -> Counter[DownloadStatus]:  # cannot be invoked directly from main because it is a coroutine.
    counter: Counter[DownloadStatus] = Counter()
    semaphore = asyncio.Semaphore(concur_req)  # Create an asyncio.Semophore that will not allow more than `concur_req` active coroutines among those using this semaphore.
    async with httpx.AsyncClient() as client:
        to_do = [download_one(client, cc, base_url, semaphore, verbose)
                 for cc in sorted(cc_list)]  # Create a list of coroutine objects, one per call
        to_do_iter = asyncio.as_completed(to_do)  # Get an iterator that will return coroutines objects as they are done.
        if not verbose:
            to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))  # Wrap as_completed iterator with the tqdm generator function to display progress
        error: httpx.HTTPError | None = None  # Declare and initialise error with None. Used to holdd an exception if one is raised
        for coro in to_do_iter:  # Iterate over the compeleted coroutine objects
            try:
                status = await coro  # this will not block becauase only has done coroutines.
            except httpx.HTTPStatusError as exc:
                error_msg = 'HTTP error {resp.status_code} - {resp.reason_phrase}'
                error_msg = error_msg.format(resp=exc.response)
                error = exc  # preserve its value
            except httpx.RequestError as exc:
                error_msg = f'{exc} {type(exc)}'.strip()
                error = exc  
            except KeyboardInterrupt:
                break

            if error:
                status = DownloadStatus.ERROR  
                if verbose:
                    url = str(error.request.url)  # extract the URL from the exception
                    cc = Path(url).stem.upper()   # extract the name of file to display the country code
                    print(f'{cc} error: {error_msg}')
            counter[status] += 1

    return counter

def download_many(cc_list: list[str],
                  base_url: str,
                  verbose: bool,
                  concur_req: int) -> Counter[DownloadStatus]:
    coro = supervisor(cc_list, base_url, verbose, concur_req)
    counts = asyncio.run(coro)  # instantiates the supervisor coroutine and passes it to the event loop

    return counts

# Making Multiple Requests for Each Download
In terminal:

    python3 flags3_asyncio.py

In [ ]:
async def get_country(client: httpx.AsyncClient,
                      base_url: str,
                      cc: str) -> str:    # coroutine returns a string with the country name
    url = f'{base_url}/{cc}/metadata.json'.lower()
    resp = await client.get(url, timeout=3.1, follow_redirects=True)
    resp.raise_for_status()
    metadata = resp.json()  # get a Python dict built from the JSON contents of the response
    return metadata['country'] 

In [ ]:
async def download_one(client: httpx.AsyncClient,
                       cc: str,
                       base_url: str,
                       semaphore: asyncio.Semaphore,
                       verbose: bool) -> DownloadStatus:
    try:
        async with semaphore:  # hold the semaphore to await for get_flag
            image = await get_flag(client, base_url, cc)
        async with semaphore:  #  same for get_country
            country = await get_country(client, base_url, cc)
    except httpx.HTTPStatusError as exc:
        res = exc.response
        if res.status_code == HTTPStatus.NOT_FOUND:
            status = DownloadStatus.NOT_FOUND
            msg = f'not found: {res.url}'
        else:
            raise
    else:
        filename = country.replace(' ', '_')  # use  country name to create a filename.
        await asyncio.to_thread(save_flag, image, f'{filename}.gif')
        status = DownloadStatus.OK
        msg = 'OK'
    if verbose and msg:
        print(cc, msg)
    return status

# Delegating Tasks to Executors

    await asyncio.to_thread(save_flag, image, f'{cc}.gif')
    
in Python 3.7 or 3.8:

    loop = asyncio.get_running_loop()
    loop.run_in_executor(None, save_flag, image, f'{cc}.gif')
    
# Writing asyncio Servers

...